In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce 
import codecs
import statsmodels.api as sm
import datetime

## Load the google trends data 

In [10]:
#Define data path 
dataset = 'Google'
DATA_FOLDER = '../raw_data/Datasets/GoogleTrends_Ontario'

print('Loading and pre-processing {} data for use in forecasting model'.format(dataset))


groups_analyzed = ['mentaldistress',
                  'financialstressors',
                  'socialstressors',
                  'treatmentseeking', 
                  'covidinformation'
                  ]


terms_included = [
                 ['depression', 'anxiety', 'suicide','fear', 'loneliness'],
                 ['eviction', 'mortgage', 'mortgageloan', 'unemployment', 'foodbank', 'welfare', 'ei', 'employmentinsurance'],
                 ['pharmacy', 'education', 'abuse', 'alcohol', 'divorce'],
                 ['cognitivebehavioraltherapy', 'cbt', 'selfcare', 'counselling', 'crisishotline', 'helpline', 'mindfulness'],
                 ['covid', 'sick', 'symptoms', 'cough', 'fever', 'flu', 'rsv']]

group_num = 0
combine_all_df = []

for current_group in groups_analyzed:
    combine_df = []
    
    for search_term in terms_included[group_num]:
        print(group_num)
        print(search_term)
#         df_in = pd.read_csv('./{}.csv'.format(search_term))
        df_in = pd.read_csv('{}/{}.csv'.format(DATA_FOLDER,search_term))

        if search_term in terms_included[group_num][0]: #If first time through
            combine_df = df_in
            combine_df.loc[1:] = df_in.iloc[1:].replace('<1', 0).astype('int64')
#             combine_df = combine_df.rename(columns={'Category: All categories':search_term})
            
            combine_data = pd.to_numeric(combine_df['Category: All categories'].iloc[1:])

        else:
            #Remove any non-numeric values
            combine_df[search_term] = df_in.iloc[1:].replace('<1', 0).astype('int64')
            
            combine_df[search_term].loc['Week'] = search_term + ': (Ontario)'

            combine_data = combine_data + pd.to_numeric(combine_df[search_term].iloc[1:])

            #Create a running sum of values in each column for total search terms


            #Format running sum of all values and save in dataframe  
            combine_data_final = combine_data.iloc[:].values
            combine_data_final = np.insert(combine_data_final, 0, 0).tolist()
            combine_data_final[0] = 'all'+current_group+': (Ontario)'

    combine_df['all'+current_group] = combine_data_final[:]

    #remove first row 
    combine_df = combine_df.iloc[1:]
    
    combine_df.to_csv('{}/combinations/all{}.csv'.format(DATA_FOLDER, current_group))
    
    if current_group == groups_analyzed[0]: #First time through
        combine_all_df = combine_df
    else:
        combine_all_df = combine_all_df.merge(combine_df, left_index=True, right_index=True, how='inner')
        
    
    group_num += 1


Loading and pre-processing Google data for use in forecasting model
0
depression
0
anxiety
0
suicide
0
fear
0
loneliness
1
eviction
1
mortgage
1
mortgageloan
1
unemployment
1
foodbank
1
welfare
1
ei
1
employmentinsurance
2
pharmacy
2
education
2
abuse
2
alcohol
2
divorce
3
cognitivebehavioraltherapy
3
cbt
3
selfcare
3
counselling
3
crisishotline
3
helpline
3
mindfulness
4
covid
4
sick


/tmp/ipykernel_22570/3631583079.py:37: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  combine_df.loc[1:] = df_in.iloc[1:].replace('<1', 0).astype('int64')
/tmp/ipykernel_22570/3631583079.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine_df[search_term].loc['Week'] = search_term + ': (Ontario)'
/tmp/ipykernel_22570/3631583079.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine_df[search_term].loc['Week'] = search_term + ': (Ontario)'
/tmp/ipykernel_22570/3631583079.py:46: Sett

4
symptoms
4
cough
4
fever
4
flu
4
rsv


/tmp/ipykernel_22570/3631583079.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine_df[search_term].loc['Week'] = search_term + ': (Ontario)'
/tmp/ipykernel_22570/3631583079.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine_df[search_term].loc['Week'] = search_term + ': (Ontario)'
/tmp/ipykernel_22570/3631583079.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine_df[search_term].loc['Week'] = search_term + 

In [11]:
combine_all_df

,Category: All categories_x,anxiety,suicide,fear,loneliness,allmentaldistress,Category: All categories_y,mortgage,mortgageloan,unemployment,...,mindfulness,alltreatmentseeking,Category: All categories,sick,symptoms,cough,fever,flu,rsv,allcovidinformation
2018-01-07,80,82.0,33.0,38.0,48.0,281.0,35,41.0,56.0,15.0,...,72.0,285.0,0,83.0,24.0,43.0,31.0,32.0,4.0,217.0
2018-01-14,100,84.0,32.0,38.0,100.0,354.0,19,48.0,70.0,14.0,...,83.0,404.0,0,86.0,23.0,41.0,34.0,33.0,2.0,219.0
2018-01-21,94,87.0,26.0,35.0,70.0,312.0,14,40.0,43.0,15.0,...,69.0,343.0,0,83.0,23.0,45.0,40.0,33.0,2.0,226.0
2018-01-28,97,92.0,27.0,35.0,43.0,294.0,32,37.0,44.0,14.0,...,83.0,385.0,0,83.0,23.0,47.0,37.0,34.0,2.0,226.0
2018-02-04,88,80.0,23.0,35.0,47.0,273.0,22,34.0,76.0,14.0,...,84.0,364.0,0,76.0,23.0,48.0,38.0,35.0,4.0,224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-29,75,81.0,18.0,42.0,47.0,263.0,43,48.0,74.0,13.0,...,60.0,350.0,4,66.0,25.0,42.0,30.0,11.0,6.0,184.0
2023-02-05,65,79.0,17.0,41.0,25.0,227.0,36,42.0,55.0,12.0,...,54.0,429.0,3,67.0,26.0,43.0,30.0,12.0,7.0,188.0
2023-02-12,65,78.0,16.0,39.0,56.0,254.0,23,38.0,70.0,10.0,...,61.0,316.0,3,58.0,25.0,42.0,30.0,11.0,4.0,173.0
2023-02-19,61,82.0,19.0,37.0,40.0,239.0,24,42.0,85.0,8.0,...,56.0,340.0,3,68.0,26.0,46.0,31.0,12.0,5.0,191.0


In [21]:
for column in combine_all_df.columns:
    if 'Category:' in column:
        print(column)
        combine_all_df = combine_all_df.drop(columns=column)
        
#Add google prefix 
combine_all_df = combine_all_df.rename(columns={old_name: 'google_' + old_name for old_name in combine_all_df.columns})

combine_all_df.to_csv('../processed_data_inputs/GoogleTrends.csv')

In [22]:
combine_all_df

,google_anxiety,google_suicide,google_fear,google_loneliness,google_allmentaldistress,google_mortgage,google_mortgageloan,google_unemployment,google_foodbank,google_welfare,...,google_helpline,google_mindfulness,google_alltreatmentseeking,google_sick,google_symptoms,google_cough,google_fever,google_flu,google_rsv,google_allcovidinformation
2018-01-07,82.0,33.0,38.0,48.0,281.0,41.0,56.0,15.0,20.0,59.0,...,36.0,72.0,285.0,83.0,24.0,43.0,31.0,32.0,4.0,217.0
2018-01-14,84.0,32.0,38.0,100.0,354.0,48.0,70.0,14.0,21.0,62.0,...,37.0,83.0,404.0,86.0,23.0,41.0,34.0,33.0,2.0,219.0
2018-01-21,87.0,26.0,35.0,70.0,312.0,40.0,43.0,15.0,18.0,80.0,...,43.0,69.0,343.0,83.0,23.0,45.0,40.0,33.0,2.0,226.0
2018-01-28,92.0,27.0,35.0,43.0,294.0,37.0,44.0,14.0,23.0,93.0,...,79.0,83.0,385.0,83.0,23.0,47.0,37.0,34.0,2.0,226.0
2018-02-04,80.0,23.0,35.0,47.0,273.0,34.0,76.0,14.0,11.0,69.0,...,54.0,84.0,364.0,76.0,23.0,48.0,38.0,35.0,4.0,224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-29,81.0,18.0,42.0,47.0,263.0,48.0,74.0,13.0,50.0,65.0,...,45.0,60.0,350.0,66.0,25.0,42.0,30.0,11.0,6.0,184.0
2023-02-05,79.0,17.0,41.0,25.0,227.0,42.0,55.0,12.0,21.0,66.0,...,47.0,54.0,429.0,67.0,26.0,43.0,30.0,12.0,7.0,188.0
2023-02-12,78.0,16.0,39.0,56.0,254.0,38.0,70.0,10.0,48.0,47.0,...,48.0,61.0,316.0,58.0,25.0,42.0,30.0,11.0,4.0,173.0
2023-02-19,82.0,19.0,37.0,40.0,239.0,42.0,85.0,8.0,35.0,52.0,...,59.0,56.0,340.0,68.0,26.0,46.0,31.0,12.0,5.0,191.0
